In [ ]:
import numpy as np
import pandas as pd

from Utils import extract_symbol_timeframe

tp = 0.5
period_rsi = 14

# === RSI (быстрый NumPy вариант) ===
def compute_rsi(prices, period=period_rsi):
    deltas = np.diff(prices)
    seed = deltas[:period]
    up = seed[seed >= 0].sum() / period
    down = -seed[seed < 0].sum() / period
    rs = up / down if down != 0 else 0
    rsi = np.zeros_like(prices)
    rsi[:period] = 100. - 100. / (1. + rs)

    for i in range(period, len(prices)):
        delta = deltas[i - 1]
        gain = max(delta, 0)
        loss = -min(delta, 0)
        up = (up * (period - 1) + gain) / period
        down = (down * (period - 1) + loss) / period
        rs = up / down if down != 0 else 0
        rsi[i] = 100. - 100. / (1. + rs)
    return rsi

# === Сделки на основе входа RSI > 30 ===
def extract_rsi_cross_trades(df, rsi, tp=tp, stop_loss_pct=99, entry_threshold=30):
    rsi_shift = np.roll(rsi, 1)
    rsi_cross = (rsi_shift < entry_threshold) & (rsi >= entry_threshold)
    rsi_cross[0] = False

    close = df["close"].to_numpy()
    high = df["high"].to_numpy()
    low = df["low"].to_numpy()
    open_time = df["open_time"].to_numpy()

    trades = []
    last_exit = 0

    for i in np.where(rsi_cross)[0]:
        if i <= last_exit:
            continue

        entry_price = close[i]
        take_profit = entry_price * (1 + tp / 100)
        stop_loss = entry_price * (1 - stop_loss_pct / 100)
        entry_time = open_time[i]

        stop_triggered = False
        stop_time = pd.NaT

        for j in range(i + 1, len(close)):
            if high[j] >= take_profit:
                exit_price = take_profit
                exit_time = open_time[j]
                break
            elif low[j] <= stop_loss:
                exit_price = stop_loss
                exit_time = open_time[j]
                stop_triggered = True
                stop_time = open_time[j]
                break
        else:
            break

        duration = int((exit_time - entry_time).total_seconds() // 60)

        trades.append({
            "entry_time": entry_time,
            "entry_price": entry_price,
            "rsi": rsi[i],
            "exit_time": exit_time,
            "exit_price": exit_price,
            "duration_min": duration,
            "stop_triggered": stop_triggered,
            "stop_time": stop_time
        })

        last_exit = j

    return pd.DataFrame(trades)

# === Загрузка данных и расчёт ===

path = " " # Format filename = {symbol}USDT_timeframe_data.csv, example: "BTCUSDT_1h_from_1_Jan_2020.csv"
df = pd.read_csv(path)

symbol, timeframe = extract_symbol_timeframe(path)
indicator_name = "RSI_simple"

df["open_time"] = pd.to_datetime(df["open_time"], utc=True)

rsi_period = 14
rsi = compute_rsi(df["close"].to_numpy(dtype=np.float64), period=rsi_period)
df = df.iloc[len(df) - len(rsi):].copy()
df["rsi"] = rsi

deals_df = extract_rsi_cross_trades(df, rsi, tp=tp, stop_loss_pct=99, entry_threshold=30)

# === Вывод ===
print("✅ Найдено сделок:", len(deals_df))
print(f"Криптовалюта: {symbol}\nТаймфрейм: {timeframe}\nСтратегия: {indicator_name}\nПериод RSI: {period_rsi}\nПрофит: {tp}%")
deals_df.tail()
